In [1]:
!pip install split-folders

In [2]:
import splitfolders

In [3]:
splitfolders.ratio('rice_leaf_diseases', output="output", seed=1337, ratio=(.8, 0.1,0.1))

Copying files: 120 files [00:00, 247.63 files/s]


In [4]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [5]:
IMAGE_SIZE=[224,224]
train_path="Dataset\train"
valid_path="Dataset\val"

In [6]:
vgg=VGG16(input_shape=IMAGE_SIZE+[3],weights="imagenet",include_top=False)

In [7]:
for layer in vgg.layers:
    layer.trainable = False

In [8]:
folders = glob('Dataset/train/*')

In [9]:
len(folders)

3

In [10]:
x = Flatten()(vgg.output)

In [11]:
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [15]:
training_set = train_datagen.flow_from_directory('Dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size =4,
                                                 class_mode = 'categorical')

Found 96 images belonging to 3 classes.


In [16]:
test_set = test_datagen.flow_from_directory('Dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 4,
                                            class_mode = 'categorical')

Found 12 images belonging to 3 classes.


In [17]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=30,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\akrit\AppData\Local\Temp/ipykernel_22424/2336048694.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/30
24/24 [==============================] - 30s 1s/step - loss: 1.3378 - accuracy: 0.5312 - val_loss: 2.3235 - val_accuracy: 0.3333
Epoch 2/30
24/24 [==============================] - 27s 1s/step - loss: 0.5413 - accuracy: 0.7604 - val_loss: 2.0874 - val_accuracy: 0.2500
Epoch 3/30
24/24 [==============================] - 27s 1s/step - loss: 0.2807 - accuracy: 0.8958 - val_loss: 2.1626 - val_accuracy: 0.4167
Epoch 4/30
24/24 [==============================] - 27s 1s/step - loss: 0.2740 - accuracy: 0.9062 - val_loss: 1.8504 - val_accuracy: 0.4167
Epoch 5/30
24/24 [==============================] - 27s 1s/step - loss: 0.1228 - accuracy: 0.9688 - val_loss: 2.1094 - val_accuracy: 0.4167
Epoch 6/30
24/24 [==============================] - 26s 1s/step - loss: 0.0873 - accuracy: 0.9583 - val_loss: 2.0608 - val_accuracy: 0.3333
Epoch 7/30
24/24 [==============================] - 26s 1s/step - loss: 0.0543 - accuracy: 1.0000 - val_loss: 1.8081 - val_accuracy: 0.4167
Epoch 8/30
24/24 [==

In [18]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
st.set_option('deprecation.showfileUploaderEncoding',False)
def load_model():
    model=tf.keras.models.load_model("mymodel2.hdf5")
    return model
model=load_model()
st.write("Leaf detection")
file=st.file_uploader("Please upload image",type=["jpg","png","jpeg"])
import cv2
from PIL import Image,ImageOps
import numpy as np
def import_and_predict(image_data,model):
    size=(224,224)
    image=ImageOps.fit(image_data,size,Image.ANTIALIAS)
    image=np.asarray(image)
    image_reshape=image[np.newaxis,...]
    prediction=model.predict(image_reshape)
if file is None:
    st.text("Please upload an image file")
else:
    image=Image.open(file)
    st.image(image,use_column_width=True)
    predictions=import_and_predict(image,model)
    class_names=['Brown Spot','Leaf smut','Bacterial leaf blight']
    strings="The disease detected in this leaf is : "+class_names[np.argmax(predictions)]
    st.success(strings)    



Overwriting app.py
